In [1]:
import torch
import numpy as np
from torchvision.models import resnet50
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_cifar10_choosen
from attack import attack,test_model
device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
train_dataloader, test_dataloader, test_dataloader_all = load_cifar10_choosen(choosen_classes=[0, 1, 2, 3])


Files already downloaded and verified
Files already downloaded and verified


In [5]:
def train_choosen_classes(train_dataloader, test_dataloader, epochs=10, lr=0.001):
    model = resnet50(pretrained=False, num_classes=10).to(device)
    loss_func = nn.CrossEntropyLoss()
    best_acc = -np.inf
    for epoch in range(epochs):
        model.train()
        all_preds = []
        all_labels = []
        for i, (x, y) in enumerate(train_dataloader):
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = loss_func(output, y)
            loss.backward()
            for param in model.parameters():
                param.data -= lr * param.grad
                param.grad.zero_()
        model.eval()
        train_correct,train_total = test_model(model, train_dataloader)
        test_correct, test_total = test_model(model, test_dataloader)
        print(f"Epoch {epoch + 1} train acc: {train_correct / train_total}")
        print(f"Epoch {epoch + 1} test acc: {test_correct / test_total}")
        if test_correct / test_total > best_acc:
            best_acc = test_correct / test_total
            torch.save(model.state_dict(), "best_model.pth")

In [ ]:
train_choosen_classes(train_dataloader, test_dataloader, epochs=10, lr=0.001)